<a href="https://colab.research.google.com/github/ctokx/blochsphere/blob/main/nb/Gemma3_(4B).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [Gemma 3 blog](https://unsloth.ai/blog/gemma3) for what's new in Unsloth and our [Reasoning blog](https://unsloth.ai/blog/r1-reasoning) on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [3]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [ ]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 03-22 01:21:18 [__init__.py:256] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.18: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update a small amount of parameters!

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


<a name="Data"></a>
### Data Prep
We now use the `Gemma-3` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. Gemma-3 renders multi turn conversations like below:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3` and more.

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

We now use `standardize_data_formats` to try converting datasets to the correct format for finetuning purposes!

In [ ]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

Let's see how row 100 looks like!

In [ ]:
dataset[100]

{'conversations': [{'content': 'What is the modulus operator in programming and how can I use it to calculate the modulus of two given numbers?',
   'role': 'user'},
  {'content': 'In programming, the modulus operator is represented by the \'%\' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:\n\n```python\n# Calculate the modulus\nModulus = a % b\n\nprint("Modulus of the given numbers is: ", Modulus)\n```\n\nIn this code snippet, the variables \'a\' and \'b\' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator \'%\', we calculate the remainder when \'a\' is divided by \'b\'. The result is then stored in the variable \'Modulus\'. Finally, the modulus value is printed using the \'print\' statement.\n\nFor example, if \'a\' is 10 and \'b\' is 4, the modulus calculation would be 10 % 4, which equals 2. Theref

We now have to apply the chat template for `Gemma-3` onto the conversations, and save it to `text`

In [ ]:
def apply_chat_template(examples):
    texts = tokenizer.apply_chat_template(examples["conversations"])
    return { "text" : texts }
pass
dataset = dataset.map(apply_chat_template, batched = True)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Let's see how the chat template did! Notice `Gemma-3` default adds a `<bos>`!

In [ ]:
dataset[100]["text"]

'<bos><start_of_turn>user\nWhat is the modulus operator in programming and how can I use it to calculate the modulus of two given numbers?<end_of_turn>\n<start_of_turn>model\nIn programming, the modulus operator is represented by the \'%\' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:\n\n```python\n# Calculate the modulus\nModulus = a % b\n\nprint("Modulus of the given numbers is: ", Modulus)\n```\n\nIn this code snippet, the variables \'a\' and \'b\' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator \'%\', we calculate the remainder when \'a\' is divided by \'b\'. The result is then stored in the variable \'Modulus\'. Finally, the modulus value is printed using the \'print\' statement.\n\nFor example, if \'a\' is 10 and \'b\' is 4, the modulus calculation would be 10 % 4, which equals 2. Therefore, t

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16
Unsloth: We found double BOS tokens - we shall remove one automatically.


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

Let's verify masking the instruction part is done! Let's print the 100th row again:

In [ ]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<bos><start_of_turn>user\nWhat is the modulus operator in programming and how can I use it to calculate the modulus of two given numbers?<end_of_turn>\n<start_of_turn>model\nIn programming, the modulus operator is represented by the \'%\' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:\n\n```python\n# Calculate the modulus\nModulus = a % b\n\nprint("Modulus of the given numbers is: ", Modulus)\n```\n\nIn this code snippet, the variables \'a\' and \'b\' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator \'%\', we calculate the remainder when \'a\' is divided by \'b\'. The result is then stored in the variable \'Modulus\'. Finally, the modulus value is printed using the \'print\' statement.\n\nFor example, if \'a\' is 10 and \'b\' is 4, the modulus calculation would be 10 % 4, which equals 2. Therefore, t

Now let's print the masked out example - you should see only the answer is present:

In [ ]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                               In programming, the modulus operator is represented by the \'%\' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:\n\n```python\n# Calculate the modulus\nModulus = a % b\n\nprint("Modulus of the given numbers is: ", Modulus)\n```\n\nIn this code snippet, the variables \'a\' and \'b\' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator \'%\', we calculate the remainder when \'a\' is divided by \'b\'. The result is then stored in the variable \'Modulus\'. Finally, the modulus value is printed using the \'print\' statement.\n\nFor example, if \'a\' is 10 and \'b\' is 4, the modulus calculation would be 10 % 4, which equals 2. Therefore, the output of the above code would be:\n\n```\nModulus of the given numbers is: 2\n```\n\nThis means that the modulus of 10 and 4 is 2.<end_of_t

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
5.457 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 14,901,248/4,000,000,000 (0.37% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.223000
2,1.672500
3,1.753200
4,1.366300
5,1.122400
6,1.443300
7,0.772900
8,1.134300
9,0.914400
10,0.814100


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

470.5985 seconds used for training.
7.84 minutes used for training.
Peak reserved memory = 6.309 GB.
Peak reserved memory for training = 0.852 GB.
Peak reserved memory % of max memory = 42.799 %.
Peak reserved memory for training % of max memory = 5.78 %.


<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "Continue the sequence: 1, 1, 2, 3, 5, 8,",
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nContinue the sequence: 1, 1, 2, 3, 5, 8,<end_of_turn>\n<start_of_turn>model\n13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 7']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "Why is the sky blue?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

The sky is blue because of a phenomenon called Rayleigh scattering. This is when sunlight is scattered by the molecules in the atmosphere. Blue light is scattered more than other colors because it has a shorter wavelength. This is why we see a blue sky when we look up at the sky.<end_of_turn>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("gemma-3")  # Local saving
tokenizer.save_pretrained("gemma-3")
# model.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving

['gemma-3/processor_config.json']

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastModel
    model, tokenizer = FastModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "What is Gemma-3?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Gemma-3 is a family of open-source language models developed by the team at Google DeepMind. They are based on the same architecture as the previous Gemma models, but with a larger context window and improved performance. Gemma-3 models are available in various sizes, from 2B to 7B parameters


### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3-finetune`. Set `if False` to `if True` to let it run!

In [ ]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "HF_ACCOUNT/gemma-3-finetune", tokenizer,
        token = "hf_..."
    )

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/gemma-finetune-gguf",
        token = "hf_...",
    )

Now, use the `gemma-3-finetune.gguf` file or `gemma-3-finetune-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>


In [1]:
# Install Unsloth and required libraries
!pip install --no-deps unsloth bitsandbytes accelerate xformers==0.0.29.post3 peft trl datasets huggingface_hub hf_transfer
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 47.2 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers (to revision v4.49.0-Gemma-3) to /tmp/pip-req-build-v15xp6gb
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-v15xp6gb
  Running command git checkout -q 367bab469b0ef32017e2a0a0a5dbac5d36002f03
  Resolved https://github.com/huggingface/transformers to commit 367bab469b0ef32017e2a0a0a5dbac5d36002f03
  Installing build de

In [6]:
from unsloth import FastLanguageModel
import torch

# Configuration
max_seq_length = 2048  # Supports RoPE scaling internally
dtype = None  # Auto-detect: Float16 or Bfloat16 based on hardware

# Load Gemma model
model_name = "unsloth/gemma-3-4b-it"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True
)

# Get token IDs for "Yes" and "No"
yes_input = tokenizer("Yes", add_special_tokens=False)
yes_token_id = yes_input["input_ids"][0]
no_input = tokenizer("No", add_special_tokens=False)
no_token_id = no_input["input_ids"][0]

# Get input embeddings
input_embeddings = model.get_input_embeddings()  # Typically model.model.embed_tokens

# Create a new linear layer for the classification head with output size 2
new_lm_head = torch.nn.Linear(input_embeddings.embedding_dim, 2, bias=False)

# Initialize the weights with the embeddings for "No" and "Yes"
new_lm_head.weight = torch.nn.Parameter(torch.vstack([
    input_embeddings.weight[no_token_id, :],
    input_embeddings.weight[yes_token_id, :]
]))

# Set the new layer as the output embeddings
model.set_output_embeddings(new_lm_head)

# Verify the new output embeddings shape
print(f"Trimmed output_embeddings shape: {new_lm_head.weight.shape}")

==((====))==  Unsloth 2025.3.18: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Trimmed output_embeddings shape: torch.Size([2, 2560])


In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Rank for LoRA
    target_modules=[
        "lm_head", "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=True
)
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

Unsloth: Making `model.base_model.model.vision_tower.vision_model.encoder` require gradients
Trainable parameters: 32829472


In [10]:
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split

# Load and split dataset
data = pd.read_csv("finance_sentiment.csv")  # Columns: text, label (0 or 1)
train_size, val_size = 1000, 1000
data_sample = data.sample(n=train_size + val_size, random_state=42)
train_df, val_df = train_test_split(data_sample, test_size=val_size/len(data_sample), random_state=42)

# Save the train DataFrame to a CSV file
train_csv_path = "train_data.csv"
train_df.to_csv(train_csv_path, index=False)

# Load the dataset using the CSV file path
train_dataset = load_dataset("csv", data_files={"train": train_csv_path})["train"]

# Define prompt
prompt = """<bos><start_of_turn>user
Here is a financial tweet:
{}

Does this tweet have a positive sentiment? Answer with "Yes" or "No".
<end_of_turn>
<start_of_turn>model
{}<end_of_turn>"""

positivelabel = "Yes"
negativelabel = "No"

# Define formatting function
def formatting_prompts_func(examples):
    texts = []
    for text, label in zip(examples["text"], examples["label"]):
        answer = positivelabel if label == 1 else negativelabel
        formatted = prompt.format(text, answer)
        texts.append(formatted)
    return {"text": texts}

# Apply the formatting function
train_dataset = train_dataset.map(formatting_prompts_func, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [23]:
from transformers import DataCollatorForLanguageModeling

class DataCollatorForLastTokenLM(DataCollatorForLanguageModeling):
    def __init__(self, *args, mlm=False, **kwargs):
        super().__init__(*args, mlm=mlm, **kwargs)
        self.ignore_index = -100  # Define ignore_index as an attribute

    def torch_call(self, examples):
        batch = super().torch_call(examples)
        for i in range(len(examples)):
            last_token_idx = (batch["labels"][i] != self.ignore_index).nonzero()[-1].item()
            batch["labels"][i, :last_token_idx] = self.ignore_index
            batch["labels"][i, last_token_idx] = 1 if batch["labels"][i, last_token_idx] == yes_token_id else 0
        return batch

# Define yes_token_id using the tokenizer (adjust token as needed)
yes_token_id = tokenizer.tokenizer.convert_tokens_to_ids("Yes")

# Initialize the collator with mlm=False
collator = DataCollatorForLastTokenLM(tokenizer=tokenizer.tokenizer, mlm=False)

In [28]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files={"train": "finance_sentiment.csv"})["train"]
train_dataset = dataset.select(range(1000))  # Sample 1000 rows as per your setup

def formatting_conversations_func(examples):
    conversations = []
    for tweet, label in zip(examples["text"], examples["label"]):
        user_message = f"Here is a financial tweet:\n{tweet}\nDoes this tweet have a positive sentiment? Answer with \"Yes\" or \"No\"."
        model_message = "Yes" if label == 1 else "No"
        conversation = [
            {"role": "user", "content": user_message},
            {"role": "model", "content": model_message}
        ]
        conversations.append(conversation)
    return {"conversations": conversations}

# Apply the formatting WITHOUT removing original columns
train_dataset = train_dataset.map(
    formatting_conversations_func,
    batched=True,
    # Don't remove the columns yet, as SFTTrainer will need 'text'
)

In [25]:
from trl import SFTTrainer, SFTConfig
from unsloth.chat_templates import train_on_responses_only

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    args=SFTConfig(
        per_device_train_batch_size=32,
        gradient_accumulation_steps=1,
        warmup_steps=10,
        learning_rate=1e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=3407,
        output_dir="outputs",
        num_train_epochs=1,
        report_to="none",
        group_by_length=True
    ),
    data_collator=collator
)

# Train on model responses only
trainer = train_on_responses_only(
    trainer,
    instruction_part="<start_of_turn>user\n",
    response_part="<start_of_turn>model\n"
)

trainer.train()

Unsloth: Switching to float32 training since model cannot work with float16


Applying chat template to train dataset (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

KeyError: "Invalid keys in the example: {'messages', 'label'}"

In [ ]:
FastLanguageModel.for_inference(model)
val_dataset = load_dataset("pandas", data_files={"val": val_df})["val"]

# Tokenize and group by length
tokenized_inputs = [
    (tokenizer(prompt.format(text, ""), return_tensors="pt"), label)
    for text, label in zip(val_df["text"], val_df["label"])
]
tokenized_inputs.sort(key=lambda x: x[0]["input_ids"].shape[1])

from collections import defaultdict
grouped_inputs = defaultdict(list)
for tokenized_input, label in tokenized_inputs:
    length = tokenized_input["input_ids"].shape[1]
    grouped_inputs[length].append((tokenized_input, label))

# Batched inference
batch_size = 64
all_preds, all_labels = [], []
from tqdm import tqdm
import torch.nn.functional as F

for length, group in tqdm(grouped_inputs.items()):
    for i in range(0, len(group), batch_size):
        batch = group[i:i + batch_size]
        input_ids = torch.cat([x[0]["input_ids"] for x in batch], dim=0).to("cuda")
        attention_mask = torch.cat([x[0]["attention_mask"] for x in batch], dim=0).to("cuda")
        labels = [x[1] for x in batch]

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits[:, -1, :]
        probs = F.softmax(logits, dim=-1)
        preds = torch.argmax(probs, dim=-1).cpu().numpy()

        all_preds.extend(preds)
        all_labels.extend(labels)

# Calculate accuracy
correct = sum(1 for p, l in zip(all_preds, all_labels) if str(p) == str(l))
accuracy = correct / len(all_labels)
print(f"Accuracy: {accuracy}")

In [ ]:
model.save_pretrained("gemma_text_classification")

In [70]:
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling
from trl import SFTTrainer, SFTConfig
from unsloth.chat_templates import train_on_responses_only
import torch

# Load and split dataset
data = pd.read_csv("finance_sentiment.csv")  # Assumes columns: text, label (0 or 1)
train_size, val_size = 100, 100
data_sample = data.sample(n=train_size + val_size, random_state=42)
train_df, val_df = train_test_split(data_sample, test_size=val_size/len(data_sample), random_state=42)

# Save to CSV and load as dataset
train_csv_path = "train_data.csv"
train_df.to_csv(train_csv_path, index=False)
train_dataset = load_dataset("csv", data_files={"train": train_csv_path})["train"]

# Define function to create conversations column
def formatting_conversations_func(example):
    tweet = example["text"]
    label = example["label"]
    user_message = f"Here is a financial tweet:\n{tweet}\nDoes this tweet have a positive sentiment? Answer with \"Yes\" or \"No\"."
    model_message = "Yes" if label == 1 else "No"
    conversation = [
        {"role": "user", "content": user_message},
        {"role": "assistant", "content": model_message}
    ]
    return {"conversations": conversation}

# Apply formatting to create the "conversations" column
train_dataset = train_dataset.map(
    formatting_conversations_func,
    remove_columns=["text", "label"]
)

# Debug Step 1: Check the dataset after adding conversations
print("=== Debug Step 1: After adding conversations ===")
print("First example:", train_dataset[0])
print("Type of 'conversations':", type(train_dataset[0]["conversations"]))

# Define function to add pre-formatted text column
def add_text_column(example):
    formatted = tokenizer.apply_chat_template(example["conversations"], tokenize=False)
    if isinstance(formatted, list):
        formatted = formatted[0]  # Ensure it’s a single string
    return {"text": str(formatted)}

# Add the "text" column to the dataset
train_dataset = train_dataset.map(add_text_column)

# Debug Step 2: Check the dataset after adding text
print("\n=== Debug Step 2: After adding text ===")
print("First example:", train_dataset[0])
print("Type of 'text':", type(train_dataset[0]["text"]))

# Define custom data collator (assuming this is part of your setup)
class DataCollatorForLastTokenLM(DataCollatorForLanguageModeling):
    def __init__(self, *args, mlm=False, **kwargs):
        super().__init__(*args, mlm=mlm, **kwargs)
        self.ignore_index = -100

    def torch_call(self, examples):
        batch = super().torch_call(examples)
        for i in range(len(examples)):
            last_token_idx = (batch["labels"][i] != self.ignore_index).nonzero()[-1].item()
            batch["labels"][i, :last_token_idx] = self.ignore_index
            batch["labels"][i, last_token_idx] = 1 if batch["labels"][i, last_token_idx] == yes_token_id else 0
        return batch

# Define yes_token_id (assumes tokenizer is defined elsewhere)
yes_token_id = tokenizer.tokenizer.convert_tokens_to_ids("Yes")

# Initialize collator
collator = DataCollatorForLastTokenLM(tokenizer=tokenizer.tokenizer, mlm=False)

# Initialize SFTTrainer
trainer = SFTTrainer(
    model=model,  # Assumes model is defined
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    args=SFTConfig(
        per_device_train_batch_size=32,
        gradient_accumulation_steps=1,
        warmup_steps=10,
        learning_rate=1e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=3407,
        output_dir="outputs",
        num_train_epochs=1,
        report_to="none",
        group_by_length=True
    ),
    data_collator=collator,
    dataset_text_field="text"  # Use the pre-formatted text column
)

# Apply train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part="<start_of_turn>user\n",
    response_part="<start_of_turn>model\n"
)

# Debug Step 3: Inspect tokenized fields
print("\n=== Debug Step 3: After train_on_responses_only ===")
for i, example in enumerate(trainer.train_dataset):
    print(f"Example {i}:")
    for key in ["input_ids", "attention_mask", "labels"]:
        value = example.get(key)
        if value is None:
            print(f"  {key}: None")
        elif isinstance(value, list):
            print(f"  {key}: list of length {len(value)}")
        else:
            print(f"  {key}: {type(value)}")
    if i >= 5:  # Limit to first few examples
        break
# Start training
trainer.train()

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

=== Debug Step 1: After adding conversations ===
First example: {'conversations': [{'content': 'Here is a financial tweet:\n$AVLR - Avalara: Buy This Leader In The Tax Automation Trend. https://t.co/lqeMIiPUxZ #economy #stockmarket #markets\nDoes this tweet have a positive sentiment? Answer with "Yes" or "No".', 'role': 'user'}, {'content': 'Yes', 'role': 'assistant'}]}
Type of 'conversations': <class 'list'>


Map:   0%|          | 0/100 [00:00<?, ? examples/s]


=== Debug Step 2: After adding text ===
First example: {'conversations': [{'content': 'Here is a financial tweet:\n$AVLR - Avalara: Buy This Leader In The Tax Automation Trend. https://t.co/lqeMIiPUxZ #economy #stockmarket #markets\nDoes this tweet have a positive sentiment? Answer with "Yes" or "No".', 'role': 'user'}, {'content': 'Yes', 'role': 'assistant'}], 'text': '<bos><start_of_turn>user\nHere is a financial tweet:\n$AVLR - Avalara: Buy This Leader In The Tax Automation Trend. https://t.co/lqeMIiPUxZ #economy #stockmarket #markets\nDoes this tweet have a positive sentiment? Answer with "Yes" or "No".<end_of_turn>\n<start_of_turn>model\nYes<end_of_turn>\n'}
Type of 'text': <class 'str'>
Unsloth: Switching to float32 training since model cannot work with float16


Converting train dataset to ChatML (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

ArrowInvalid: cannot mix list and non-list, non-null values

In [64]:
for ex in train_dataset[:5]:
    tokenized = tokenizer(sft_formatting_func(ex))
    print(type(tokenized["input_ids"]), len(tokenized["input_ids"]))

TypeError: string indices must be integers, not 'str'

In [71]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling
from trl import SFTTrainer, SFTConfig
from unsloth.chat_templates import train_on_responses_only

# Configuration
max_seq_length = 2048  # Supports RoPE scaling internally
dtype = None  # Auto-detect: Float16 or Bfloat16 based on hardware

# Load Gemma model with 4-bit quantization
model_name = "unsloth/gemma-3-4b-it"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True
)

# Apply LoRA for efficient fine-tuning
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Rank for LoRA
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=True
)

print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

==((====))==  Unsloth 2025.3.18: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Making `model.base_model.model.vision_tower.vision_model.encoder` require gradients
Trainable parameters: 32788480


In [83]:
# Load and sample dataset
data = pd.read_csv("finance_sentiment.csv")  # Assumes columns: text, label (0 or 1)
train_size, val_size = 1000, 1000
data_sample = data.sample(n=train_size + val_size, random_state=42)
train_df, val_df = train_test_split(data_sample, test_size=val_size/len(data_sample), random_state=42)

# Save training data to CSV and load as dataset
train_csv_path = "train_data.csv"
train_df.to_csv(train_csv_path, index=False)
train_dataset = load_dataset("csv", data_files={"train": train_csv_path})["train"]

Generating train split: 0 examples [00:00, ? examples/s]

In [84]:
# Define prompt and formatting function
prompt = """Here is a financial tweet:
{}

Does this tweet have a positive sentiment? Answer with "Yes" or "No".
SOLUTION
The correct answer is: "{}"""

def formatting_prompts_func(example):
    tweet = example["text"]
    label = "Yes" if example["label"] == 1 else "No"
    formatted = prompt.format(tweet, label)
    return {"text": formatted}

# Apply formatting to add the prompt
train_dataset = train_dataset.map(
    formatting_prompts_func,
    remove_columns=["text", "label"]
)

# Debug: Check dataset after formatting
print("=== After formatting prompts ===")
print("First example:", train_dataset[0])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

=== After formatting prompts ===
First example: {'text': 'Here is a financial tweet:\nTesla\'s stock falls 2.1% premarket, after tumbling 17.2% on Wednesday\n\nDoes this tweet have a positive sentiment? Answer with "Yes" or "No".\nSOLUTION\nThe correct answer is: "No'}


In [85]:
# Custom data collator to focus on the last token
class DataCollatorForResponseTokenLM(DataCollatorForLanguageModeling):
    def __init__(self, *args, mlm=False, **kwargs):
        super().__init__(*args, mlm=mlm, **kwargs)
        self.ignore_index = -100

    def torch_call(self, examples):
        batch = super().torch_call(examples)
        for i in range(len(examples)):
            labels = batch["labels"][i]
            # Find the last non-ignored token (the classification token)
            last_token_idx = (labels != self.ignore_index).nonzero(as_tuple=True)[0][-1].item()
            # Set all labels before the last token to -100
            batch["labels"][i, :last_token_idx] = self.ignore_index
        return batch

# Initialize collator
collator = DataCollatorForResponseTokenLM(tokenizer=tokenizer, mlm=False)

In [86]:
# Initialize SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    args=SFTConfig(
        per_device_train_batch_size=32,
        gradient_accumulation_steps=1,
        warmup_steps=10,
        learning_rate=1e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=3407,
        output_dir="outputs",
        num_train_epochs=1,
        report_to="none",
        group_by_length=True
    ),
    data_collator=collator,
    dataset_text_field="text"  # Use the pre-formatted text column
)

# Train the model
trainer.train()

Unsloth: Switching to float32 training since model cannot work with float16


Converting train dataset to ChatML (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

ArrowInvalid: cannot mix list and non-list, non-null values